## imports

In [67]:
import polars as pl
from polars import col as c
import os, sys, requests
from dotenv import load_dotenv, find_dotenv
from tqdm import tqdm

load_dotenv(find_dotenv())

True

## constants

In [68]:
OMDB_API_KEY = os.getenv("OMDB_API_KEY")
FILES_DIR = "../data"

## load df

In [69]:
file_name = "movies_df.xlsx"
file_path = os.path.join(FILES_DIR, file_name)
movies_df = pl.read_excel(file_path)

Could not determine dtype for column 2, falling back to string
Could not determine dtype for column 3, falling back to string


## code

In [70]:
movies_df.head()

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles,years,titles,cleaned_title,cleaned_year,ids
i64,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str
1,"""Действуй сестра""",null,null,"""Not found""","""Act sister""",null,"""Not found""",null,"""Sister Act""","""1992""","""Sister Act""","""Sister Act""",1992,"""tt0105417"""
2,"""Summer lovers""",null,null,"""Summer Lovers_1982_Comedy, Dra…","""Summer lovers""",1982,"""Summer Lovers""","""checked""","""Summer lovers""",null,null,"""Summer Lovers""",1982,"""tt0084737"""
3,"""Джон уик""",null,null,"""John Wick_2014_Action, Crime, …","""John Wick""",2014,"""John Wick""","""checked""","""Джон уик""",null,null,"""John Wick""",2014,"""tt2911666"""
4,"""Помни""",null,null,"""Remembered_2015_Short, Drama_N…","""Remembered""",2015,"""Remembered""","""checked""","""Помни""",null,null,"""Memento""",2000,"""tt0209144"""
5,"""Похитители велосипедов""",null,null,"""Not found""","""Cycles of bicycles""",null,"""Not found""",null,"""Bicycle Thieves""","""1948""","""Bicycle Thieves""","""Bicycle Thieves""",1948,"""tt0040522"""


In [71]:
movies_df.filter(c("ids") == "Not found")

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles,years,titles,cleaned_title,cleaned_year,ids
i64,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str
193,"""True detective *""",null,null,null,null,null,null,"""checked""","""True detective *""",null,null,null,null,"""Not found"""
355,"""(100 любимых фильмов фрэнка оу…",null,null,"""Not found""","""(100 favorite films Frank Ocea…",null,"""Not found""",null,"""(100 Favorite Films of Frank O…","""-1""",null,null,null,"""Not found"""
450,"""Отбросы *""",null,null,"""Not found""","""Dispers *""",null,"""Not found""",null,"""Misfits""","""2009–2013""","""Misfits""","""Misfits""",null,"""Not found"""
493,"""Мanhattan shorts *""","""2020""",null,"""Not found""","""Мanhattan shorts *""",null,"""Not found""",null,"""Manhattan Shorts""","""-1""",null,null,null,"""Not found"""


In [72]:
def fetch_english_title(title):
    try:
        url = f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&i={requests.utils.quote(title)}"  # for index search
        # url = f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&t={requests.utils.quote(title)}"
        response = requests.get(url)
        data = response.json()
        if data.get("Response") == "True" and "Title" in data:
            return data
        else:
            print(f"Not found in OMDb: {title}")
            return "Not found"
    except Exception as e:
        print(f"OMDb error for {title}: {e}")
        return "Not found"

In [73]:
fetch_english_title(movies_df.head(1)["ids"].item())

{'Title': 'Sister Act',
 'Year': '1992',
 'Rated': 'PG',
 'Released': '29 May 1992',
 'Runtime': '100 min',
 'Genre': 'Comedy, Family, Music',
 'Director': 'Emile Ardolino',
 'Writer': 'Paul Rudnick',
 'Actors': 'Whoopi Goldberg, Maggie Smith, Harvey Keitel',
 'Plot': 'When a nightclub singer is forced to take refuge from the mob in a convent, she ends up turning the convent choir into a soulful chorus complete with a Motown repertoire, until the sudden celebrity of the choir jeopardizes her ide...',
 'Language': 'English',
 'Country': 'United States',
 'Awards': '9 wins & 10 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjIzOTU0ODItMzcyMi00YzFjLTk3MzktNTQ2YjZmN2E0NTBkXkEyXkFqcGc@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.5/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '73%'},
  {'Source': 'Metacritic', 'Value': '51/100'}],
 'Metascore': '51',
 'imdbRating': '6.5',
 'imdbVotes': '113,562',
 'imdbID': 'tt0105417',
 'Type': 

In [76]:
result_dict = dict()
for row in tqdm(movies_df.iter_rows(named=True)):
    if row["ids"] == "Not found":
        result_dict[row["index"]] = "Not found"
    else:
        omdb_result = fetch_english_title(row["ids"])
        if omdb_result != "Not found":
            result_dict[row["index"]] = omdb_result
        else:
            result_dict[row["index"]] = "Not found"

622it [03:54,  2.66it/s]


In [77]:
for i, (key, val) in enumerate(result_dict.items()):
    print(f"{key}\t{val}")
    if i > 9:
        break

1	{'Title': 'Sister Act', 'Year': '1992', 'Rated': 'PG', 'Released': '29 May 1992', 'Runtime': '100 min', 'Genre': 'Comedy, Family, Music', 'Director': 'Emile Ardolino', 'Writer': 'Paul Rudnick', 'Actors': 'Whoopi Goldberg, Maggie Smith, Harvey Keitel', 'Plot': 'When a nightclub singer is forced to take refuge from the mob in a convent, she ends up turning the convent choir into a soulful chorus complete with a Motown repertoire, until the sudden celebrity of the choir jeopardizes her ide...', 'Language': 'English', 'Country': 'United States', 'Awards': '9 wins & 10 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjIzOTU0ODItMzcyMi00YzFjLTk3MzktNTQ2YjZmN2E0NTBkXkEyXkFqcGc@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.5/10'}, {'Source': 'Rotten Tomatoes', 'Value': '73%'}, {'Source': 'Metacritic', 'Value': '51/100'}], 'Metascore': '51', 'imdbRating': '6.5', 'imdbVotes': '113,562', 'imdbID': 'tt0105417', 'Type': 'movie', 'DVD': 'N/A'

In [136]:
creds = []
indexes = result_dict.keys()
for key in indexes:
    creds.append(result_dict[key])

In [137]:
parsed_films = pl.from_dict(
    {
        "index": indexes,
        "creds": creds,
    }
)

parsed_films.sample(5)

index,creds
i64,struct[25]
101,"{""I, Robot"",""2004"",""PG-13"",""16 Jul 2004"",""115 min"",""Action, Mystery, Sci-Fi"",""Alex Proyas"",""Jeff Vintar, Akiva Goldsman, Isaac Asimov"",""Will Smith, Bridget Moynahan, Bruce Greenwood"",""In 2035, a technophobic cop investigates a crime that may have been perpetrated by a robot, which leads to a larger threat to humanity."",""English"",""United States, Germany"",""Nominated for 1 Oscar. 1 win & 15 nominations total"",""https://m.media-amazon.com/images/M/MV5BZDdhNTY3YTgtYmQwMC00MjM1LTgzYzMtMGM1N2E0NWM1NDlkXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""7.1/10""}, {""Metacritic"",""59/100""}],""59"",""7.1"",""598,159"",""tt0343818"",""movie"",""08 Jun 2004"",""$144,801,023"",""N/A"",""N/A"",""True""}"
61,"{""Lockout"",""2012"",""PG-13"",""13 Apr 2012"",""95 min"",""Action, Sci-Fi, Thriller"",""James Mather, Steve Saint Leger"",""James Mather, Steve Saint Leger, Luc Besson"",""Guy Pearce, Maggie Grace, Peter Stormare"",""A man wrongfully accused and convicted of conspiring to commit espionage against the US government is offered his freedom if he can rescue the president's daughter from an outer-space prison that's been taken over by violent inmates."",""English"",""France, United States"",""1 nomination total"",""https://m.media-amazon.com/images/M/MV5BMTg1OTQ3MzM0NV5BMl5BanBnXkFtZTcwNDU1MDI0Nw@@._V1_SX300.jpg"",[{""Internet Movie Database"",""6.0/10""}, {""Rotten Tomatoes"",""38%""}, {""Metacritic"",""48/100""}],""48"",""6.0"",""102,329"",""tt1592525"",""movie"",""N/A"",""$14,326,864"",""N/A"",""N/A"",""True""}"
543,"{""Reptile"",""2023"",""R"",""29 Sep 2023"",""134 min"",""Crime, Drama, Mystery"",""Grant Singer"",""Grant Singer, Benjamin Brewer, Benicio Del Toro"",""Benicio Del Toro, Justin Timberlake, Eric Bogosian"",""Tom Nichols is a hardened New England detective, unflinching in his pursuit of a case where nothing is as it seems and it begins to dismantle the illusions in his own life."",""English"",""United States"",""N/A"",""https://m.media-amazon.com/images/M/MV5BN2ZkOGI0ZDgtMTJlZi00Y2QyLTg2YWEtNmI2MzcwYWIwYzUwXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""6.8/10""}, {""Rotten Tomatoes"",""44%""}, {""Metacritic"",""52/100""}],""52"",""6.8"",""99,860"",""tt13274016"",""movie"",""N/A"",""N/A"",""N/A"",""N/A"",""True""}"
295,"{""Vertigo"",""1958"",""PG"",""28 May 1958"",""128 min"",""Mystery, Romance, Thriller"",""Alfred Hitchcock"",""Alec Coppel, Samuel A. Taylor, Pierre Boileau"",""James Stewart, Kim Novak, Barbara Bel Geddes"",""A former San Francisco police detective juggles wrestling with his personal demons and becoming obsessed with the hauntingly beautiful woman he has been hired to trail, who may be deeply disturbed."",""English"",""United States"",""Nominated for 2 Oscars. 9 wins & 8 nominations total"",""https://m.media-amazon.com/images/M/MV5BNzEyZTUxMWItMGVhYy00Y2NlLTljYTMtYzBmMzI1YmUyMDBkXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""8.2/10""}, {""Rotten Tomatoes"",""93%""}, {""Metacritic"",""100/100""}],""100"",""8.2"",""448,438"",""tt0052357"",""movie"",""N/A"",""$7,863,310"",""N/A"",""N/A"",""True""}"
308,"{""Kong: Skull Island"",""2017"",""PG-13"",""10 Mar 2017"",""118 min"",""Action, Adventure, Fantasy"",""Jordan Vogt-Roberts"",""Dan Gilroy, Max Borenstein, Derek Connolly"",""Tom Hiddleston, Samuel L. Jackson, Brie Larson"",""After the Vietnam war, a team of scientists explores an uncharted island in the Pacific, venturing into the domain of the mighty Kong and must fight to escape a primal Eden."",""English, Vietnamese, Japanese"",""United States, China, Vietnam, Canada, Germany, United Kingdom, Singapore, India, Australia"",""Nominated for 1 Oscar. 1 win & 22 nominations total"",""https://m.media-amazon.com/images/M/MV5BMTcwMjcxYTQtYjJlOS00YTE1LWIyZGYtOGRlOTBlMWZmYTliXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""6.7/10""}, {""Rotten Tomatoes"",""75%""}, {""Metacritic"",""62/100""}],""62"",""6.7"",""3

In [153]:
movies_df.head()

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles,years,titles,cleaned_title,cleaned_year,ids
i64,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str
1,"""Действуй сестра""",null,null,"""Not found""","""Act sister""",null,"""Not found""",null,"""Sister Act""","""1992""","""Sister Act""","""Sister Act""",1992,"""tt0105417"""
2,"""Summer lovers""",null,null,"""Summer Lovers_1982_Comedy, Dra…","""Summer lovers""",1982,"""Summer Lovers""","""checked""","""Summer lovers""",null,null,"""Summer Lovers""",1982,"""tt0084737"""
3,"""Джон уик""",null,null,"""John Wick_2014_Action, Crime, …","""John Wick""",2014,"""John Wick""","""checked""","""Джон уик""",null,null,"""John Wick""",2014,"""tt2911666"""
4,"""Помни""",null,null,"""Remembered_2015_Short, Drama_N…","""Remembered""",2015,"""Remembered""","""checked""","""Помни""",null,null,"""Memento""",2000,"""tt0209144"""
5,"""Похитители велосипедов""",null,null,"""Not found""","""Cycles of bicycles""",null,"""Not found""",null,"""Bicycle Thieves""","""1948""","""Bicycle Thieves""","""Bicycle Thieves""",1948,"""tt0040522"""


In [139]:
movies_df = movies_df.drop("creds")

In [140]:
movies_df = movies_df.join(parsed_films, on="index", how="left")

In [131]:
movies_df.head()

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles,years,titles,cleaned_title,cleaned_year,ids,creds
i64,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str,struct[25]
1,"""Действуй сестра""",null,null,"""Not found""","""Act sister""",null,"""Not found""",null,"""Sister Act""","""1992""","""Sister Act""","""Sister Act""",1992,"""tt0105417""","{""Sister Act"",""1992"",""PG"",""29 May 1992"",""100 min"",""Comedy, Family, Music"",""Emile Ardolino"",""Paul Rudnick"",""Whoopi Goldberg, Maggie Smith, Harvey Keitel"",""When a nightclub singer is forced to take refuge from the mob in a convent, she ends up turning the convent choir into a soulful chorus complete with a Motown repertoire, until the sudden celebrity of the choir jeopardizes her ide..."",""English"",""United States"",""9 wins & 10 nominations total"",""https://m.media-amazon.com/images/M/MV5BMjIzOTU0ODItMzcyMi00YzFjLTk3MzktNTQ2YjZmN2E0NTBkXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""6.5/10""}, {""Rotten Tomatoes"",""73%""}, {""Metacritic"",""51/100""}],""51"",""6.5"",""113,562"",""tt0105417"",""movie"",""N/A"",""$139,605,150"",""N/A"",""N/A"",""True""}"
2,"""Summer lovers""",null,null,"""Summer Lovers_1982_Comedy, Dra…","""Summer lovers""",1982,"""Summer Lovers""","""checked""","""Summer lovers""",null,null,"""Summer Lovers""",1982,"""tt0084737""","{""Summer Lovers"",""1982"",""R"",""16 Jul 1982"",""98 min"",""Comedy, Drama, Romance"",""Randal Kleiser"",""Randal Kleiser"",""Peter Gallagher, Daryl Hannah, Valérie Quennessen"",""A young American couple and a French woman engage in a threesome in the Greek Islands."",""English"",""United States"",""N/A"",""https://m.media-amazon.com/images/M/MV5BMTYxNjk0Mjg1NF5BMl5BanBnXkFtZTcwNzAyMTM3NA@@._V1_SX300.jpg"",[{""Internet Movie Database"",""5.8/10""}, {""Rotten Tomatoes"",""25%""}],""N/A"",""5.8"",""4,687"",""tt0084737"",""movie"",""N/A"",""$4,968,000"",""N/A"",""N/A"",""True""}"
3,"""Джон уик""",null,null,"""John Wick_2014_Action, Crime, …","""John Wick""",2014,"""John Wick""","""checked""","""Джон уик""",null,null,"""John Wick""",2014,"""tt2911666""","{""John Wick"",""2014"",""R"",""24 Oct 2014"",""101 min"",""Action, Crime, Thriller"",""Chad Stahelski"",""Derek Kolstad"",""Keanu Reeves, Michael Nyqvist, Alfie Allen"",""John Wick is a former hitman grieving the loss of his true love. When his home is broken into, robbed, and his dog killed, he is forced to return to action to exact revenge."",""English, Russian, Hungarian"",""United States"",""5 wins & 10 nominations total"",""https://m.media-amazon.com/images/M/MV5BMTU2NjA1ODgzMF5BMl5BanBnXkFtZTgwMTM2MTI4MjE@._V1_SX300.jpg"",[{""Internet Movie Database"",""7.5/10""}, {""Rotten Tomatoes"",""86%""}, {""Metacritic"",""68/100""}],""68"",""7.5"",""783,681"",""tt2911666"",""movie"",""N/A"",""$43,037,835"",""N/A"",""N/A"",""True""}"
4,"""Помни""",null,null,"""Remembered_2015_Short, Drama_N…","""Remembered""",2015,"""Remembered""","""checked""","""Помни""",null,null,"""Memento""",2000,"""tt0209144""","{""Memento"",""2000"",""R"",""25 May 2001"",""113 min"",""Drama, Mystery, Thriller"",""Christopher Nolan"",""Christopher Nolan, Jonathan Nolan"",""Guy Pearce, Carrie-Anne Moss, Joe Pantoliano"",""A former insurance investigator who now suffers from anterograde amnesia uses notes and tattoos to hunt down his wife's murderer."",""English"",""United States"",""Nominated for 2 Oscars. 57 wins & 59 nominations total"",""https://m.media-amazon.com/images/M/MV5BYmQ3MjliNjAtNWFiZS00YWI1LTlmZTktMzBiNDE1NjRhZjU0XkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""8.4/10""}, {""Rotten Tomatoes"",""94%""}, {""Metacritic"",""83/100""}],""83"",""8.4"",""1,389,289"",""tt0209144"",""movie"",""N/A"",""$25,544,867"",""N/A"",""N/A"",""True""}"
5,"""Похитители велосипедов""",null,null,"""Not found""","""Cycles of bicycles""",null,"""Not found""",null,"""Bicycle Thieves""","""1948""","""Bicycle Thieves""","""Bicycle Thieves""",

In [112]:
movies_df.head(1)["creds"].item()

{'Title': 'Sister Act',
 'Year': '1992',
 'Rated': 'PG',
 'Released': '29 May 1992',
 'Runtime': '100 min',
 'Genre': 'Comedy, Family, Music',
 'Director': 'Emile Ardolino',
 'Writer': 'Paul Rudnick',
 'Actors': 'Whoopi Goldberg, Maggie Smith, Harvey Keitel',
 'Plot': 'When a nightclub singer is forced to take refuge from the mob in a convent, she ends up turning the convent choir into a soulful chorus complete with a Motown repertoire, until the sudden celebrity of the choir jeopardizes her ide...',
 'Language': 'English',
 'Country': 'United States',
 'Awards': '9 wins & 10 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjIzOTU0ODItMzcyMi00YzFjLTk3MzktNTQ2YjZmN2E0NTBkXkEyXkFqcGc@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.5/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '73%'},
  {'Source': 'Metacritic', 'Value': '51/100'}],
 'Metascore': '51',
 'imdbRating': '6.5',
 'imdbVotes': '113,562',
 'imdbID': 'tt0105417',
 'Type': 

In [148]:
movies_df.with_columns(
    c("creds").struct.field("Year").cast(pl.Int64).alias("year_creds"),
    c("creds").struct.field("Title").alias("title_creds"),
    c("creds").struct.field("Genre").alias("genre_creds"),
).filter(
    # c("year_creds") != c("cleaned_year"),
    c("genre_creds").is_null(),
)

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles,years,titles,cleaned_title,cleaned_year,ids,creds,year_creds,title_creds,genre_creds
i64,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str,struct[25],i64,str,str
193,"""True detective *""",null,null,null,null,null,null,"""checked""","""True detective *""",null,null,null,null,"""Not found""","{null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null}",null,null,null
355,"""(100 любимых фильмов фрэнка оу…",null,null,"""Not found""","""(100 favorite films Frank Ocea…",null,"""Not found""",null,"""(100 Favorite Films of Frank O…","""-1""",null,null,null,"""Not found""","{null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null}",null,null,null
450,"""Отбросы *""",null,null,"""Not found""","""Dispers *""",null,"""Not found""",null,"""Misfits""","""2009–2013""","""Misfits""","""Misfits""",null,"""Not found""","{null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null}",null,null,null
493,"""Мanhattan shorts *""","""2020""",null,"""Not found""","""Мanhattan shorts *""",null,"""Not found""",null,"""Manhattan Shorts""","""-1""",null,null,null,"""Not found""","{null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null}",null,null,null
null,"""The Big Sleep""","""1946""","""2026-07-30 00:00:00""",null,null,null,null,null,null,null,null,"""The Big Sleep""",1946,"""tt0038355""",null,null,null,null


### to replace

In [ ]:
# movies_df =
movies_df.with_columns(
    pl.when(c("index") == "tt0038355")
    .then(pl.lit("tt0110413"))
    .otherwise(c("ids"))
    .alias("ids")
)

In [ ]:
omdb_result = fetch_english_title("tt0110413")
if omdb_result != "Not found":
    # result_dict[9] = omdb_result
    pass
else:
    print(1)

In [ ]:
# to replace
to_replace = {
    63: "tt2024469",
}

In [ ]:
def fetch_english_title(title):
    try:
        url = f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&i={requests.utils.quote(title)}"  # for index search
        # url = f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&t={requests.utils.quote(title)}"
        response = requests.get(url)
        data = response.json()
        if data.get("Response") == "True" and "Title" in data:
            return data
        else:
            print(f"Not found in OMDb: {title}")
            return "Not found"
    except Exception as e:
        print(f"OMDb error for {title}: {e}")
        return "Not found"

In [ ]:
# movies_df =
for key, val in to_replace.items():
    movies_df = movies_df.with_columns(
        pl.when(c("index") == key).then(pl.lit(val)).otherwise(c("ids")).alias("ids")
    )
    omdb_result = fetch_english_title(val)

    movies_df = movies_df.with_columns(
        pl.when(c("index") == key)
        .then(omdb_result)
        .otherwise(c("creds"))
        .alias("creds")
    )

## save df

In [150]:
file_name = "movies_df.xlsx"
file_path = os.path.join(FILES_DIR, file_name)
movies_df.write_excel(file_path)

In [151]:
file_name = "movies_df.parquet"
file_path = os.path.join(FILES_DIR, file_name)
movies_df.write_parquet(file_path)